In [10]:
# Importar las librerías necesarias
import pandas as pd
import datetime
from mlxtend.preprocessing import TransactionEncoder
from mlxtend.frequent_patterns import apriori, association_rules


##### INTR. ANÁLISIS DE DATOS (PRIMER SEMESTRE)

<p style="color: #007BA7; font-size: 50px;">REGLAS DE ASOCIACIÓN: DATASET PACIENTES CON CÁNCER DE PULMÓN</p>

**Felipe Sepúlveda, Diego Müller**



<div style="text-align: center;">
<img src="https://sciencemediacentre.es/sites/default/files/2023-04/c%C3%A1ncer%20pulm%C3%B3n%20ok.jpg" alt="..." width="720"/>
</div>

---

### CONTEXTO DEL CONJUNTO DE DATOS
Este conjunto de datos contiene información sobre la mortalidad por cáncer de pulmón. La base de datos es una colección de **información sobre pacientes diagnosticados con cáncer**. Está diseñada para el análisis de diversos factores que pueden influir en el pronóstico del cáncer y los resultados del tratamiento. La base de datos incluye **variables demográficas, médicas y relacionadas con el tratamiento, con detalles sobre la condición y la historia de cada paciente**.

### Columnas del conjunto de datos
- **id**: Un identificador único para cada paciente en el conjunto de datos.
- **age**: La edad del paciente en el momento del diagnóstico.
- **gender**: El género del paciente (por ejemplo, masculino, femenino).
- **country**: El país o región donde reside el paciente.
- **diagnosis_date**: La fecha en que se diagnosticó al paciente con cáncer de pulmón.
- **cancer_stage**: El estadio del cáncer de pulmón en el momento del diagnóstico (por ejemplo, Estadio I, Estadio II, Estadio III, Estadio IV).
- **family_history**: Indica si hay antecedentes familiares de cáncer (por ejemplo, sí, no).
- **smoking_status**: El estado de fumador del paciente (por ejemplo, fumador actual, exfumador, nunca fumó, fumador pasivo).
- **bmi**: El Índice de Masa Corporal del paciente en el momento del diagnóstico.
- **cholesterol_level**: El nivel de colesterol del paciente (valor).
- **hypertension**: Indica si el paciente tiene hipertensión (presión arterial alta) (por ejemplo, sí, no).
- **asthma**: Indica si el paciente tiene asma (por ejemplo, sí, no).
- **cirrhosis**: Indica si el paciente tiene cirrosis hepática (por ejemplo, sí, no).
- **other_cancer**: Indica si el paciente ha tenido algún otro tipo de cáncer además del diagnóstico principal (por ejemplo, sí, no).
- **treatment_type**: El tipo de tratamiento que recibió el paciente (por ejemplo, cirugía, quimioterapia, radiación, combinado).
- **end_treatment_date**: La fecha en que el paciente completó su tratamiento contra el cáncer o falleció.
- **survived**: Indica si el paciente sobrevivió (por ejemplo, sí, no).



In [13]:
# Se cargan datos 
ubi= 'lung_cancer_mortality_data_small.csv'

df = pd.read_csv(ubi)
df



,id,age,gender,country,diagnosis_date,cancer_stage,family_history,smoking_status,bmi,cholesterol_level,hypertension,asthma,cirrhosis,other_cancer,treatment_type,end_treatment_date,survived
0,1,64.0,Male,Croatia,2016-04-05,Stage I,Yes,Current Smoker,27.3,196,0,1,0,0,Radiation,2018-01-09,0
1,2,50.0,Female,Italy,2023-04-20,Stage III,No,Passive Smoker,22.4,234,1,1,1,0,Chemotherapy,2023-11-28,0
2,3,65.0,Male,Slovakia,2023-04-05,Stage IV,No,Former Smoker,20.2,210,0,0,0,0,Chemotherapy,2025-01-12,0
3,4,51.0,Female,Greece,2016-02-05,Stage III,Yes,Never Smoked,41.8,262,1,0,1,0,Surgery,2016-11-14,0
4,5,37.0,Female,Slovakia,2023-11-29,Stage III,Yes,Passive Smoker,33.5,262,0,0,0,0,Chemotherapy,2025-03-10,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
55995,55996,49.0,Female,Germany,2014-11-15,Stage III,Yes,Never Smoked,23.6,155,0,0,0,0,Surgery,2016-02-13,0
55996,55997,65.0,Male,Luxembourg,2016-03-13,Stage IV,Yes,Current Smoker,19.6,185,0,0,0,0,Combined,2017-11-11,0
55997,55998,60.0,Female,Latvia,2023-05-21,Stage II,Yes,Passive Smoker,33.5,261,0,0,0,0,Radiation,2024-12-04,1
55998,55999,63.0,Female,Bulgaria,2015-12-09,Stage III,No,Former Smoker,24.0,221,0,0,0,0,Radiation,2017-05-10,0


In [14]:
# Cambiar nombre de item 0/1 
df['hypertension'] = df['hypertension'].replace({1: 'tiene hipertensión', 0: 'no tiene hipertensión'})
df['asthma'] = df['asthma'].replace({1: 'si tiene asma', 0: 'no tiene asma'})
df['cirrhosis'] = df['cirrhosis'].replace({1: 'tiene cirrosis', 0: 'no tiene cirrosis'})
df['other_cancer'] = df['other_cancer'].replace({1: 'tiene otro cáncer', 0: 'no tiene otro cáncer'})
df['survived'] = df['survived'].replace({1: 'si sobrevivió', 0: 'no sobrevivió'})

In [15]:
#Columnas de interés
df2_items = df[['survived', 'gender','country', 'cancer_stage','smoking_status','hypertension','asthma','cirrhosis','other_cancer','treatment_type']]
df2_items

,survived,gender,country,cancer_stage,smoking_status,hypertension,asthma,cirrhosis,other_cancer,treatment_type
0,no sobrevivió,Male,Croatia,Stage I,Current Smoker,no tiene hipertensión,si tiene asma,no tiene cirrosis,no tiene otro cáncer,Radiation
1,no sobrevivió,Female,Italy,Stage III,Passive Smoker,tiene hipertensión,si tiene asma,tiene cirrosis,no tiene otro cáncer,Chemotherapy
2,no sobrevivió,Male,Slovakia,Stage IV,Former Smoker,no tiene hipertensión,no tiene asma,no tiene cirrosis,no tiene otro cáncer,Chemotherapy
3,no sobrevivió,Female,Greece,Stage III,Never Smoked,tiene hipertensión,no tiene asma,tiene cirrosis,no tiene otro cáncer,Surgery
4,no sobrevivió,Female,Slovakia,Stage III,Passive Smoker,no tiene hipertensión,no tiene asma,no tiene cirrosis,no tiene otro cáncer,Chemotherapy
...,...,...,...,...,...,...,...,...,...,...
55995,no sobrevivió,Female,Germany,Stage III,Never Smoked,no tiene hipertensión,no tiene asma,no tiene cirrosis,no tiene otro cáncer,Surgery
55996,no sobrevivió,Male,Luxembourg,Stage IV,Current Smoker,no tiene hipertensión,no tiene asma,no tiene cirrosis,no tiene otro cáncer,Combined
55997,si sobrevivió,Female,Latvia,Stage II,Passive Smoker,no tiene hipertensión,no tiene asma,no tiene cirrosis,no tiene otro cáncer,Radiation
55998,no sobrevivió,Female,Bulgaria,Stage III,Former Smoker,no tiene hipertensión,no tiene asma,no tiene cirrosis,no tiene otro cáncer,Radiation


In [16]:
# paso todos los datos a un mismo tipo de dato
df3 = df2_items.astype(str)
df3

,survived,gender,country,cancer_stage,smoking_status,hypertension,asthma,cirrhosis,other_cancer,treatment_type
0,no sobrevivió,Male,Croatia,Stage I,Current Smoker,no tiene hipertensión,si tiene asma,no tiene cirrosis,no tiene otro cáncer,Radiation
1,no sobrevivió,Female,Italy,Stage III,Passive Smoker,tiene hipertensión,si tiene asma,tiene cirrosis,no tiene otro cáncer,Chemotherapy
2,no sobrevivió,Male,Slovakia,Stage IV,Former Smoker,no tiene hipertensión,no tiene asma,no tiene cirrosis,no tiene otro cáncer,Chemotherapy
3,no sobrevivió,Female,Greece,Stage III,Never Smoked,tiene hipertensión,no tiene asma,tiene cirrosis,no tiene otro cáncer,Surgery
4,no sobrevivió,Female,Slovakia,Stage III,Passive Smoker,no tiene hipertensión,no tiene asma,no tiene cirrosis,no tiene otro cáncer,Chemotherapy
...,...,...,...,...,...,...,...,...,...,...
55995,no sobrevivió,Female,Germany,Stage III,Never Smoked,no tiene hipertensión,no tiene asma,no tiene cirrosis,no tiene otro cáncer,Surgery
55996,no sobrevivió,Male,Luxembourg,Stage IV,Current Smoker,no tiene hipertensión,no tiene asma,no tiene cirrosis,no tiene otro cáncer,Combined
55997,si sobrevivió,Female,Latvia,Stage II,Passive Smoker,no tiene hipertensión,no tiene asma,no tiene cirrosis,no tiene otro cáncer,Radiation
55998,no sobrevivió,Female,Bulgaria,Stage III,Former Smoker,no tiene hipertensión,no tiene asma,no tiene cirrosis,no tiene otro cáncer,Radiation


In [17]:
# se crea un arreglo llamada transacciones 
transacciones=df3.values.tolist()
transacciones

[['no sobrevivió',
  'Male',
  'Croatia',
  'Stage I',
  'Current Smoker',
  'no tiene hipertensión',
  'si tiene asma',
  'no tiene cirrosis',
  'no tiene otro cáncer',
  'Radiation'],
 ['no sobrevivió',
  'Female',
  'Italy',
  'Stage III',
  'Passive Smoker',
  'tiene hipertensión',
  'si tiene asma',
  'tiene cirrosis',
  'no tiene otro cáncer',
  'Chemotherapy'],
 ['no sobrevivió',
  'Male',
  'Slovakia',
  'Stage IV',
  'Former Smoker',
  'no tiene hipertensión',
  'no tiene asma',
  'no tiene cirrosis',
  'no tiene otro cáncer',
  'Chemotherapy'],
 ['no sobrevivió',
  'Female',
  'Greece',
  'Stage III',
  'Never Smoked',
  'tiene hipertensión',
  'no tiene asma',
  'tiene cirrosis',
  'no tiene otro cáncer',
  'Surgery'],
 ['no sobrevivió',
  'Female',
  'Slovakia',
  'Stage III',
  'Passive Smoker',
  'no tiene hipertensión',
  'no tiene asma',
  'no tiene cirrosis',
  'no tiene otro cáncer',
  'Chemotherapy'],
 ['no sobrevivió',
  'Female',
  'Denmark',
  'Stage III',
  'Pass

In [18]:
te = TransactionEncoder()

In [19]:
te2 = te.fit_transform(transacciones)
te2


array([[False, False, False, ..., False, False, False],
       [False, False, False, ...,  True,  True, False],
       [False, False, False, ..., False, False, False],
       ...,
       [False, False, False, ..., False, False, False],
       [False, False,  True, ..., False, False, False],
       [False, False, False, ..., False,  True, False]])

In [ ]:
df4 = pd.DataFrame(te2, columns=te.columns_)

# Mostrar el DataFrame resultante para verificar la transformación
df4

,Austria,Belgium,Bulgaria,Chemotherapy,Combined,Croatia,Current Smoker,Cyprus,Czech Republic,Denmark,...,no sobrevivió,no tiene asma,no tiene cirrosis,no tiene hipertensión,no tiene otro cáncer,si sobrevivió,si tiene asma,tiene cirrosis,tiene hipertensión,tiene otro cáncer
0,False,False,False,False,False,True,True,False,False,False,...,True,False,True,True,True,False,True,False,False,False
1,False,False,False,True,False,False,False,False,False,False,...,True,False,False,False,True,False,True,True,True,False
2,False,False,False,True,False,False,False,False,False,False,...,True,True,True,True,True,False,False,False,False,False
3,False,False,False,False,False,False,False,False,False,False,...,True,True,False,False,True,False,False,True,True,False
4,False,False,False,True,False,False,False,False,False,False,...,True,True,True,True,True,False,False,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
55995,False,False,False,False,False,False,False,False,False,False,...,True,True,True,True,True,False,False,False,False,False
55996,False,False,False,False,True,False,True,False,False,False,...,True,True,True,True,True,False,False,False,False,False
55997,False,False,False,False,False,False,False,False,False,False,...,False,True,True,True,True,True,False,False,False,False
55998,False,False,True,False,False,False,False,False,False,False,...,True,True,True,True,True,False,False,False,False,False


In [21]:
# Calcular los ítems frecuentes utilizando el algoritmo Apriori
# Utilizamos la función apriori para encontrar ítems frecuentes en las transacciones
# - df: El DataFrame que contiene las transacciones transformadas
# - min_support=0.1: Definimos el soporte mínimo como 0.1 (10%). Esto significa que un ítem debe aparecer en al menos el 10% de las transacciones para ser considerado frecuente.
# - use_colnames=True: Usamos los nombres de las columnas (los nombres de los productos) en lugar de índices numéricos
frequent_itemsets = apriori(df4, min_support=0.2, use_colnames=True)

# Mostrar los ítems frecuentes y su soporte
print("Ítems frecuentes encontrados:")
frequent_itemsets

Ítems frecuentes encontrados:


,support,itemsets
0,0.252000,(Chemotherapy)
1,0.248196,(Combined)
2,0.251000,(Current Smoker)
3,0.500232,(Female)
4,0.249518,(Former Smoker)
...,...,...
114,0.392821,"(tiene hipertensión, no tiene otro cáncer, no ..."
115,0.218821,"(tiene hipertensión, no tiene cirrosis, si tie..."
116,0.262161,"(tiene hipertensión, no tiene otro cáncer, si ..."
117,0.249482,"(tiene hipertensión, no tiene asma, no tiene o..."


In [22]:
# Generar las reglas de asociación
# Utilizamos la función association_rules para generar las reglas a partir de los ítems frecuentes.
# - frequent_itemsets: contiene los ítems frecuentes calculados con el algoritmo Apriori.
# - metric: definimos la métrica "confidence" para evaluar las reglas.
# - min_threshold: establecemos un umbral mínimo de 0.5 (50%) para la métrica de confianza.
rules = association_rules(frequent_itemsets, metric="confidence", min_threshold=0.6)

# Filtrar y mostrar las reglas
# Seleccionamos las columnas más relevantes para mostrar: antecedentes, consecuentes, soporte, confianza y lift.
rules = rules[['antecedents', 'consequents', 'support', 'confidence', 'lift']]
print("Reglas de asociación generadas:")
rules


Reglas de asociación generadas:


,antecedents,consequents,support,confidence,lift
0,(Chemotherapy),(no tiene otro cáncer),0.230196,0.913478,1.001228
1,(Combined),(no tiene otro cáncer),0.225411,0.908195,0.995438
2,(Current Smoker),(no tiene otro cáncer),0.229518,0.914414,1.002254
3,(Female),(no sobrevivió),0.389554,0.778746,1.001096
4,(Female),(no tiene cirrosis),0.385250,0.770142,0.999768
...,...,...,...,...,...
215,"(tiene hipertensión, si tiene asma, no tiene o...",(no tiene cirrosis),0.252589,0.750916,0.974808
216,"(tiene hipertensión, si tiene asma, no tiene c...",(no tiene otro cáncer),0.252589,0.902277,0.988952
217,"(si tiene asma, no tiene cirrosis, no tiene ot...",(tiene hipertensión),0.252589,0.793637,1.063832
218,"(tiene hipertensión, si tiene asma)","(no tiene cirrosis, no tiene otro cáncer)",0.252589,0.677280,0.959879


In [23]:
# Filtrar y mostrar las reglas
# Seleccionamos solo las columnas relevantes para mostrar
rules = rules[['antecedents', 'consequents', 'support', 'confidence', 'lift']]
print("Reglas de asociación generadas:")
rules
rules.to_csv('data2.csv', index=False)


Reglas de asociación generadas:


In [24]:
# Interpretación del resultado
# Iterar sobre cada regla para imprimir una interpretación más clara
for idx, rule in rules.iterrows():
    # Convertir los conjuntos de antecedentes y consecuentes en cadenas de texto
    antecedents = ', '.join(list(rule['antecedents']))
    consequents = ', '.join(list(rule['consequents']))
    support = rule['support']
    confidence = rule['confidence']
    lift = rule['lift']
    
    # Imprimir la interpretación de cada regla
    print(f"Regla: {antecedents} -> {consequents}")
    print(f"  Soporte: {support:.2f}")
    print(f"  Confianza: {confidence:.2f}")
    print(f"  Lift: {lift:.2f}")
    print(f"  Interpretación: Si un PERSONA/ESTA ES/EN {antecedents}, hay una confianza del {confidence:.2%} de que también SEA/TENGA {consequents}. El lift de {lift:.2f} indica que esta relación es {lift:.2f} veces más probable que si los HECHOS fueran independientes.\n")


Regla: Chemotherapy -> no tiene otro cáncer
  Soporte: 0.23
  Confianza: 0.91
  Lift: 1.00
  Interpretación: Si un PERSONA/ESTA ES/EN Chemotherapy, hay una confianza del 91.35% de que también SEA/TENGA no tiene otro cáncer. El lift de 1.00 indica que esta relación es 1.00 veces más probable que si los HECHOS fueran independientes.

Regla: Combined -> no tiene otro cáncer
  Soporte: 0.23
  Confianza: 0.91
  Lift: 1.00
  Interpretación: Si un PERSONA/ESTA ES/EN Combined, hay una confianza del 90.82% de que también SEA/TENGA no tiene otro cáncer. El lift de 1.00 indica que esta relación es 1.00 veces más probable que si los HECHOS fueran independientes.

Regla: Current Smoker -> no tiene otro cáncer
  Soporte: 0.23
  Confianza: 0.91
  Lift: 1.00
  Interpretación: Si un PERSONA/ESTA ES/EN Current Smoker, hay una confianza del 91.44% de que también SEA/TENGA no tiene otro cáncer. El lift de 1.00 indica que esta relación es 1.00 veces más probable que si los HECHOS fueran independientes.

Re

### **HIPÓTESIS**: ¿UN INDIVIDUO CON ASMA SOBREVIVE A UN CÁNCER DE PULMÓN?

#### REGLAS DE ASOCIACIÓN

![...](asmasobr.png)

**Regla: tiene asma -> no sobrevivió**
- **Soporte**: 0.37
- **Confianza**: 0.78
- **Lift**: 1.00
- **Interpretación**: Si una persona tiene asma, hay una confianza del 77.78% de que también no sobrevivió. El lift de 1.00 indica que esta relación es 1.00 veces más probable que si los hechos fueran independientes.

